## Initalise

In [1]:
# Set full page width
from IPython.core.display import HTML
HTML("""
<style>
.container {
    width: 100%;
}
</style>
""")

In [2]:
import graphlab as gl
gl.canvas.set_target('ipynb')
import datetime
import os as os

%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

# for large number of cores in a machine
gl.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 32)

2016-05-02 21:57:53,636 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1462226271.log


This non-commercial license of GraphLab Create is assigned to kevin.mcisaac@gmail.com and will expire on November 06, 2016. For commercial licensing options, visit https://dato.com/buy/.


/home/ec2-user/anaconda2/envs/dato-env/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Load Data and save in fast binary format

The training data contains 37.6M rows. 8% are bookings, the remainder are clicks. The test data contains 2.5M rows where all are bookings.

Missing data
- 47k search checkin and check out dates. These are dropped.
- 167k rows are missing d1-d149 from destinations
- 36% of the distances are missing

In [3]:
train = gl.SFrame('Data/train.csv')

Finished parsing file /home/ec2-user/Notebooks/ExpediaHotelRecommendations/Data/train.csv

Parsing completed. Parsed 100 lines in 2.34207 secs.

Read 485475 lines. Lines per second: 169427

Read 2426603 lines. Lines per second: 298350

Read 4367955 lines. Lines per second: 325674

...

Read 33480380 lines. Lines per second: 353088

Read 35420132 lines. Lines per second: 353587

Finished parsing file /home/ec2-user/Notebooks/ExpediaHotelRecommendations/Data/train.csv

Parsing completed. Parsed 37670293 lines in 105.361 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,int,int,int,float,int,int,int,int,str,str,int,int,int,int,int,int,int,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [5]:
test = gl.SFrame('Data/test.csv')
destinations = gl.SFrame('Data/destinations.csv')

print "Train:", len(train), "Destinations:", len(destinations)

Finished parsing file /home/ec2-user/Notebooks/ExpediaHotelRecommendations/Data/train.csv

Parsing completed. Parsed 100 lines in 15.7038 secs.

Read 485475 lines. Lines per second: 30541.2

Read 971386 lines. Lines per second: 31329.3

Read 1456470 lines. Lines per second: 31601.4

...

Read 23569 lines. Lines per second: 2260.04

Read 47134 lines. Lines per second: 2630.59

Finished parsing file /home/ec2-user/Notebooks/ExpediaHotelRecommendations/Data/destinations.csv

Parsing completed. Parsed 62106 lines in 18.0374 secs.


Inferred types from first 100 line(s) of file as 
column_type_hints=[int,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float]
If parsing fails due to incorre

In [6]:
train.save('Data/train_raw')
test.save('Data/test_raw')
destinations.save('Data/destinations')

## Grouping/reduction

In [5]:
train = gl.SFrame('Data/train_raw')

print "Train:", len(train)

In [10]:
others = set(train.column_names()) - set(['user_id', 'srch_destination_id', 'hotel_cluster']) - set(['is_booking', 'cnt'])

others = ['hotel_continent', 'hotel_country', 'hotel_market', 'hotel_cluster']
aggs = {f:gl.aggregate.SELECT_ONE(f) for f in others}

aggs.update({'bookings':gl.aggregate.SUM('is_booking'), 'count':gl.aggregate.COUNT(), 'cnt':gl.aggregate.SUM('cnt')})

In [11]:
aggs

{'bookings': ('__builtin__sum__', ['is_booking']),
 'cnt': ('__builtin__sum__', ['cnt']),
 'count': ('__builtin__count__', ['']),
 'hotel_cluster': ('__builtin__select_one__', ['hotel_cluster']),
 'hotel_continent': ('__builtin__select_one__', ['hotel_continent']),
 'hotel_country': ('__builtin__select_one__', ['hotel_country']),
 'hotel_market': ('__builtin__select_one__', ['hotel_market'])}

In [12]:
summary = train.groupby(['user_id', 'srch_destination_id', 'hotel_cluster'],aggs )
summary

RuntimeError: Runtime Exception. Output columns names are not unique

In [22]:
summary['score'] = summary['bookings'] + summary['count']*0.1 + summary['cnt'] *0.1
summary

hotel_cluster,srch_destination_id,user_id,bookings,count,cnt,score
8,24890,28693,0,1,1,0.2
12,26663,1003225,0,3,3,0.6
86,181,1174805,0,1,3,0.4
96,25331,286482,0,1,1,0.2
61,21804,989877,0,1,1,0.2
64,17930,800177,0,1,1,0.2
70,8267,852256,0,1,1,0.2
5,8808,990746,1,2,5,1.7
73,11923,580733,0,1,3,0.4
34,8250,633737,0,4,8,1.2


In [26]:
summary.save('Data/summary')

In [27]:
features = set(summary.column_names())
features = features - set(['hotel_cluster', 'date_time', 'srch_co', 'srch_ci'])
print list(features)

['count', 'srch_destination_id', 'user_id', 'cnt', 'bookings', 'score']


In [28]:
train_small, rest = summary.random_split(fraction=0.01, seed=1983)
valdate_small, rest = summary.random_split(fraction=0.0001, seed=1983)

In [29]:
print len(train_small), len(valdate_small)

211206 2075


In [30]:
model = gl.classifier.create(train_small, target='hotel_cluster', features=features)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 200757

Number of classes           : 100

Number of feature columns   : 6

...

| 10        | 15       | 1.000000  | 11.106201    | 0.026719          | 0.026988            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: BoostedTreesClassifier, RandomForestClassifier, DecisionTreeClassifier, LogisticClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.
PROGRESS: Model selection based on validation accuracy:
PROGRESS: ---------------------------------------------
PROGRESS: BoostedTreesClassifier          : 0.0858455300331
PROGRESS: RandomForestClassifier          : 0.0757009983063
PROGRESS: DecisionTreeClassifier          : 0.0700545310974
PROGRESS: LogisticClassifier              : 0.026988
PROGRESS: ---------------------------------------------
PROGRESS: Selecting BoostedTreesClassifier based on validation set performance.


# Feature Engineering

In [3]:
train = gl.SFrame('Data/train_raw')
print "Train:", len(train)

In [6]:
# drop rows with blank check in or out dates
train['srch_co'] = train['srch_co'].apply(lambda d: None if d == '' else d)
train['srch_ci'] = train['srch_ci'].apply(lambda d: None if d == '' else d)
train = train.dropna(columns=['srch_ci', 'srch_co'], how='any')

#conver dates to datetime format to simplify feature engineering
train['date_time'] = train['date_time'].str_to_datetime(str_format='%Y-%m-%d %H:%M:%S')
train['srch_co'] = train['srch_ci'].str_to_datetime(str_format='%Y-%m-%d')
train['srch_ci'] = train['srch_ci'].str_to_datetime(str_format='%Y-%m-%d')

#split dates  into components
train.add_columns(train['date_time'].split_datetime(column_name_prefix='date_time',limit=['year', 'month', 'day', 'hour', 'weekday']));
train.add_columns(train['srch_co'].split_datetime(column_name_prefix='srch_co',limit=['year', 'month', 'day', 'weekday']))
train.add_columns(train['srch_ci'].split_datetime(column_name_prefix='srch_ci',limit=['year', 'month', 'day', 'weekday']));

#calculate days till check in and duration of stay
train['days_till_ci'] = train.apply(lambda row: (row['srch_ci'] - row['date_time']).days)
train['days_stay'] = train.apply(lambda row: (row['srch_co'] - row['srch_ci']).days)

train.save('Data/train_fe')

In [24]:
#joining the data with destinations  massively increases the size of the data set, making it hard to work with 
#train = train.join(destinations, on='srch_destination_id', how='left')

# Modeling 

In [31]:
train = gl.SFrame('Data/train_fe')

In [6]:
features = set(train.column_names())
features = features - set(['hotel_cluster', 'date_time', 'srch_co', 'srch_ci'])
print list(features)

['site_name', 'user_location_country', 'srch_co.day', 'srch_adults_cnt', 'posa_continent', 'srch_ci.day', 'hotel_country', 'user_location_region', 'hotel_continent', 'srch_destination_id', 'date_time.month', 'user_id', 'date_time.year', 'srch_co.weekday', 'srch_destination_type_id', 'is_mobile', 'srch_ci.year', 'channel', 'srch_rm_cnt', 'days_till_ci', 'srch_children_cnt', 'days_stay', 'date_time.weekday', 'cnt', 'date_time.day', 'srch_ci.month', 'orig_destination_distance', 'srch_co.month', 'srch_ci.weekday', 'is_booking', 'date_time.hour', 'srch_co.year', 'is_package', 'hotel_market', 'user_location_city']


In [ ]:
train_small, rest = train.random_split(fraction=0.01, seed=1983)
valdate_small, rest = rest.random_split(fraction=0.0001, seed=1983)

In [22]:
print len(train_small), len(valdate_small)

375638 3773


### Spot check
Boosted tree wins

In [13]:
model = gl.classifier.create(train_small, target='hotel_cluster', features=features)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 356920

Number of classes           : 100

Number of feature columns   : 35

...

| 1         | 15.450956    | 0.135958          | 4.063796          | 0.123731            | 4.099455            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

2016-05-01 06:18:53,560 [ERROR] graphlab.toolkits._main, 66: Toolkit error: Missing value (None) encountered in column 'orig_destination_distance'. Use the SFrame's dropna function to drop rows with 'None' values in them.


PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: BoostedTreesClassifier, RandomForestClassifier, DecisionTreeClassifier, LogisticClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.


ToolkitError: Missing value (None) encountered in column 'orig_destination_distance'. Use the SFrame's dropna function to drop rows with 'None' values in them.

## Boosted Tree

In [16]:
model = gl.boosted_trees_classifier.create(train_small, target='hotel_cluster', features=features)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 356826

Number of classes           : 100

Number of feature columns   : 35

...

| 8         | 124.815967   | 0.165170          | 3.463477          | 0.142196            | 3.577396            |

| 9         | 140.327488   | 0.166448          | 3.450088          | 0.142143            | 3.579726            |

| 10        | 155.716307   | 0.169466          | 3.398598          | 0.143153            | 3.538574            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



In [17]:
model.save('Models/model')

In [74]:
pred = model.predict_topk(valdate_small[0:5],output_type = 'rank', k=5)
pred

id,class,rank
0,41,0
0,73,1
0,64,2
0,91,3
0,1,4
1,91,0
1,28,1
1,48,2
1,94,3
1,1,4


In [79]:
def create_predictions(test):
    k = 5
    pred = model.predict_topk(test, output_type = 'rank', k=k)
    pred = pred.groupby('id', {'hotel_cluster':gl.aggregate.CONCAT('rank', 'class')})
    pred['hotel_cluster'] = pred['hotel_cluster'].apply(lambda d: ' '.join([str(d[r]) for r in range(0,k)]))
    return pred

In [80]:
create_predictions(valdate_small[0:5])

id,hotel_cluster
0,41 73 64 91 1
1,91 28 48 94 1
3,91 48 95 33 1
2,80 92 26 52 1
4,62 36 58 46 1


In [20]:
import ml_metrics

ImportError: No module named ml_metrics

# More features

In [ ]:
train['score'] = train['is_booking'] + train['cnt'] *0.1

In [32]:
test.column_names()

['id',
 'date_time',
 'site_name',
 'posa_continent',
 'user_location_country',
 'user_location_region',
 'user_location_city',
 'orig_destination_distance',
 'user_id',
 'is_mobile',
 'is_package',
 'channel',
 'srch_ci',
 'srch_co',
 'srch_adults_cnt',
 'srch_children_cnt',
 'srch_rm_cnt',
 'srch_destination_id',
 'srch_destination_type_id',
 'hotel_continent',
 'hotel_country',
 'hotel_market']